<div style="width: 30%; float: right; margin: 10px; margin-right: 5%;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/FHNW_Logo.svg/2560px-FHNW_Logo.svg.png" width="500" style="float: left; filter: invert(50%);"/>
</div>

<h1 style="text-align: left; margin-top: 10px; float: left; width: 60%;">
    npr Mini-Challenge 2 <br> 
</h1>

<p style="clear: both; text-align: left;">
    Bearbeitet durch Si Ben Tran, Yannic Lais, Rami Tarabishi im HS 2023.<br>Bachelor of Science FHNW in Data Science.
</p>


# Aufgabe aus dem Space
Mini-Challenge 2-C (LE6):
Chatbot

You build a simple chatbot for hotel recommendation (here you can combine with npr and webscraping challenge) or extend the chatbot from the library (auxilio, please take contact with me), with rasa or dialogflow. You can also propose a use case. Important is the way intent are recognized and NER or variables are extracted. Delivery is a report on the use case, interactions and how intents are recognized and variables extracted. Also an error analysis on concrete conversations (confidence of models, explanation of predictions, etc.) should be undertaken. The submission can be done in a group of 2 or 3.


# 1 Libraries

In [1]:
%load_ext autoreload
%autoreload 2
from preprocessing import tokenize, stem, bag_of_words

import numpy as np
import json
from tqdm import tqdm 


import nltk
# nltk.download('punkt')

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader


# 2 Preprocessor 

## 2.1 Test Tokenizer

In [2]:
word_list = ["believe", "believed", "believes", "believing", "believer"]
print(word_list)
stemmed_words = [stem(w) for w in word_list]
print(stemmed_words)

['believe', 'believed', 'believes', 'believing', 'believer']
['believ', 'believ', 'believ', 'believ', 'believ']


## 2.2 Test Stemmer

In [3]:
word_list = ["believe", "believed", "believes", "believing", "believer"]
print(word_list)
stemmed_words = [stem(w) for w in word_list]
print(stemmed_words)

['believe', 'believed', 'believes', 'believing', 'believer']
['believ', 'believ', 'believ', 'believ', 'believ']


## 2.3 Bag of Words

In [4]:
sentence = ["hello", "how", "are", "you"]
words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
bag = bag_of_words(sentence, words)
print(bag)

[0. 1. 0. 1. 0. 0. 0.]


# 3 Load Data

In [5]:
with open("intents.json", "r") as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents["intents"]:
    tag = intent["tag"]
    # add to tag list
    tags.append(tag)
    for pattern in intent["patterns"]:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

print(all_words)
print(tags)
print(xy)

['Hi', 'Hey', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Whats', 'up', 'How', 'are', 'you', 'doing', 'Howdy', 'Hi', 'there', 'Hola', 'Good', 'morning', 'Good', 'afternoon', 'Good', 'evening', 'It', 'is', 'nice', 'to', 'meet', 'you', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Have', 'a', 'nice', 'day', 'See', 'you', '!', 'Later', '!', 'Goodbye', '!', 'I', "'m", 'off', '!', 'That', "'s", 'all', 'for', 'now', '.', 'See', 'you', 'soon', '!', 'I', "'ve", 'got', 'to', 'get', 'going', '.', 'Bye', '!', 'It', 'was', 'nice', 'talking', 'to', 'you', '.', 'See', 'you', 'soon', '.', 'I', "'ve", 'got', 'to', 'get', 'going', '.', 'Have', 'a', 'great', 'day', '!', 'It', 'was', 'nice', 'talking', 'to', 'you', '.', 'Have', 'a', 'great', 'day', '!', 'I', "'ve", 'got', 'to', 'get', 'going', '.', 'Have', 'a', 'nice', 'day', '!', 'It', 'was', 'nice', 'talking', 'to', 'you', '.', 'Have', 'a', 'nice', 'day', '

# 3.1 Preprocessing Intent Data

In [6]:
# stem and lower each word and remove duplicates and sort
ignore_words = ["?", "!", ".", ","]
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

82 patterns
7 tags: ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']
107 unique stemmed words: ["'m", "'s", "'ve", 'a', 'accept', 'afternoon', 'all', 'amex', 'ani', 'anyon', 'appl', 'are', 'awesom', 'bitcoin', 'bunch', 'bye', 'can', 'card', 'cash', 'chat', 'convers', 'credit', 'crypto', 'cryptocurr', 'day', 'deliveri', 'discov', 'do', 'doe', 'dogecoin', 'ethereum', 'even', 'everyth', 'for', 'funni', 'get', 'go', 'good', 'goodby', 'googl', 'got', 'great', 'have', 'hello', 'help', 'hey', 'hi', 'hola', 'how', 'howdi', 'i', 'in', 'info', 'is', 'it', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'meet', 'morn', 'my', 'next', 'nice', 'now', 'of', 'off', 'offer', 'onli', 'order', 'packag', 'pay', 'paypal', 'product', 'sale', 'samsung', 'see', 'sell', 'ship', 'shipment', 'someth', 'soon', 'stock', 'take', 'talk', 'tell', 'thank', 'that', 'the', 'there', 'till', 'time', 'to', 'up', 'visa', 'wa', 'what', 'when', 'which', 'with', 'you', 'yo

## 3.2 Train and Test Data

In [7]:
X_train = []
y_train = []
for pattern_sentence, tag in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

print(X_train.shape)
print(X_train)
print(y_train.shape)
print(y_train)

(82, 107)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
(82,)
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 6 6 6 6
 6 6 6 6 6 6 6 6 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 0 0 0 0
 0 0 0 1 1 1 1 1]


## 3.3 Create DataLoader 

In [8]:
class ChatDataset(Dataset):
    def __init__(self, X_train, y_train):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [9]:
# define Batch size
batch_size = 2

dataset = ChatDataset(X_train=X_train, y_train=y_train)

print(dataset.n_samples)
print(dataset.x_data)
print(dataset.y_data)

# create DataLoader
train_loader = DataLoader(
    dataset=dataset, batch_size=batch_size, shuffle=True, num_workers=0
)

# get first batch
for first_batch in train_loader:
    break
# get first training batch
first_batch_inputs = first_batch[0]
first_batch_labels = first_batch[1]
print(first_batch_inputs)
print(first_batch_labels)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

82
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 1. 0.]]
[3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 6 6 6 6
 6 6 6 6 6 6 6 6 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 0 0 0 0
 0 0 0 1 1 1 1 1]
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0.,

# 4 Model

In [10]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out


# 4.1 Define Hyperparameters

In [11]:
# Hyper-parameters
batch_size = batch_size
num_epochs = 100
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)


107 7


## 4.2 Model Instanzieren

In [12]:
neural_net = NeuralNet(input_size, hidden_size, output_size).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(neural_net.parameters(), lr=learning_rate)

# 5 Train Model

In [13]:
# Train the model
for epoch in tqdm(range(num_epochs), desc="Training Progress", unit="epoch"):
    # Wrap the train_loader with tqdm to create a progress bar
    for words, labels in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = neural_net(words)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:  # Print loss after every 10 epochs
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


print(f"final loss: {loss.item():.4f}")


Training Progress:   0%|          | 0/100 [00:00<?, ?epoch/s]

Training Progress: 100%|██████████| 100/100 [00:07<00:00, 12.71epoch/s]

Epoch [100/100], Loss: 0.0009
final loss: 0.0009


## 5.1 Save Model

In [15]:
data = {
    "model_state": neural_net.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags,
}

FILE = "data.pth"
torch.save(data, FILE)

print(f"training complete. file saved to {FILE}")

training complete. file saved to data.pth


# 6 Load Model

In [16]:
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

neural_net = NeuralNet(input_size, hidden_size, output_size).to(device)
neural_net.load_state_dict(model_state)
neural_net.eval()

bot_name = "Stupid Bounty Hunter Bot"
print("Let's chat! type 'quit' to exit")
while True:
    # sentence = "do you use credit cards?"
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = neural_net(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

    if prob.item() > 0.5:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                print(f"{bot_name}: {np.random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! type 'quit' to exit
Bounty Hunter: I do not understand...
Bounty Hunter: I do not understand...
Bounty Hunter: We sell coffee and tea
Bounty Hunter: Hello, how can I help you?
Bounty Hunter: I do not understand...
Bounty Hunter: I do not understand...
Bounty Hunter: We have a wide range of coffee and tea
Bounty Hunter: I do not understand...
